In [2]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch

In [ ]:
conn_string="dbname='ftds' host='localhost' user='postgres' password='scs1638' port=5432"
conn=db.connect(conn_string)

df=pd.read_sql("select * from table_g7", conn)

In [3]:
df = pd.read_csv('P2G7_Samuel_data_raw.csv')
df

,Timestamp,Choose your gender,Age,What is your course?,Your current year of Study,What is your CGPA?,Marital status,Do you have Depression?,Do you have Anxiety?,Do you have Panic attack?,Did you seek any specialist for a treatment?
0,8/7/2020 12:02,Female,18.0,Engineering,year 1,3.00 - 3.49,No,Yes,No,Yes,No
1,8/7/2020 12:04,Male,21.0,Islamic education,year 2,3.00 - 3.49,No,No,Yes,No,No
2,8/7/2020 12:05,Male,19.0,BIT,Year 1,3.00 - 3.49,No,Yes,Yes,Yes,No
3,8/7/2020 12:06,Female,22.0,Laws,year 3,3.00 - 3.49,Yes,Yes,No,No,No
4,8/7/2020 12:13,Male,23.0,Mathemathics,year 4,3.00 - 3.49,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...
96,13/07/2020 19:56:49,Female,21.0,BCS,year 1,3.50 - 4.00,No,No,Yes,No,No
97,13/07/2020 21:21:42,Male,18.0,Engineering,Year 2,3.00 - 3.49,No,Yes,Yes,No,No
98,13/07/2020 21:22:56,Female,19.0,Nursing,Year 3,3.50 - 4.00,Yes,Yes,No,Yes,No
99,13/07/2020 21:23:57,Female,23.0,Pendidikan Islam,year 4,3.50 - 4.00,No,No,No,No,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 11 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Timestamp                                     101 non-null    object 
 1   Choose your gender                            101 non-null    object 
 2   Age                                           100 non-null    float64
 3   What is your course?                          101 non-null    object 
 4   Your current year of Study                    101 non-null    object 
 5   What is your CGPA?                            101 non-null    object 
 6   Marital status                                101 non-null    object 
 7   Do you have Depression?                       101 non-null    object 
 8   Do you have Anxiety?                          101 non-null    object 
 9   Do you have Panic attack?                     101 non-null    obj

In [7]:
df.isnull().sum()

Timestamp                                       0
Choose your gender                              0
Age                                             1
What is your course?                            0
Your current year of Study                      0
What is your CGPA?                              0
Marital status                                  0
Do you have Depression?                         0
Do you have Anxiety?                            0
Do you have Panic attack?                       0
Did you seek any specialist for a treatment?    0
dtype: int64

In [16]:
df = df.dropna()
df['Age'] = df['Age'].astype(int)

In [17]:
def normalcolumn(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ','_')
    df.columns = df.columns.str.replace('?','')
    df.rename(columns={'choose_your_gender': 'gender', 'what_is_your_course': 'subject', 'your_current_year_of_study': 'current_year_study', 
                   'what_is_your_cgpa': 'cgpa', 'do_you_have_depression': 'have_depression', 'do_you_have_panic_attack': 'panic_attack',
                   'do_you_have_anxiety': 'have_anxiety', 'did_you_seek_any_specialist_for_a_treatment': 'specialist_treatment'}, inplace=True)

normalcolumn(df)
df.columns

C:\Users\Chris\AppData\Local\Temp\ipykernel_4972\3100133760.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('?','')


Index(['timestamp', 'gender', 'age', 'subject', 'current_year_study', 'cgpa',
       'marital_status', 'have_depression', 'have_anxiety', 'panic_attack',
       'specialist_treatment'],
      dtype='object')

In [18]:
df['cgpa'] = df['cgpa'].str.strip()
df['current_year_study'] = df['current_year_study'].str.lower()
df['subject'] = df['subject'].str.strip()
df['subject'] = df['subject'].str.lower()
df['subject'] = df['subject'].str.replace('islamic education', 'pendidikan Islam')

In [19]:
df

,timestamp,gender,age,subject,current_year_study,cgpa,marital_status,have_depression,have_anxiety,panic_attack,specialist_treatment
0,8/7/2020 12:02,Female,18,engineering,year 1,3.00 - 3.49,No,Yes,No,Yes,No
1,8/7/2020 12:04,Male,21,pendidikan Islam,year 2,3.00 - 3.49,No,No,Yes,No,No
2,8/7/2020 12:05,Male,19,bit,year 1,3.00 - 3.49,No,Yes,Yes,Yes,No
3,8/7/2020 12:06,Female,22,laws,year 3,3.00 - 3.49,Yes,Yes,No,No,No
4,8/7/2020 12:13,Male,23,mathemathics,year 4,3.00 - 3.49,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...
96,13/07/2020 19:56:49,Female,21,bcs,year 1,3.50 - 4.00,No,No,Yes,No,No
97,13/07/2020 21:21:42,Male,18,engineering,year 2,3.00 - 3.49,No,Yes,Yes,No,No
98,13/07/2020 21:22:56,Female,19,nursing,year 3,3.50 - 4.00,Yes,Yes,No,Yes,No
99,13/07/2020 21:23:57,Female,23,pendidikan islam,year 4,3.50 - 4.00,No,No,No,No,No


In [ ]:
def save_data(df_clean):
    df_clean.to_csv(('P2G7_Samuel_data_clean.csv'), index=False)

def import_data(df_clean):
    es = Elasticsearch("http://localhost:9200") 
    for i,r in df_clean.iterrows():
        doc=r.to_json()
        res=es.index(index="test_gc7", body=doc)